# QAOA grid search v27

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [14]:
import qiskit
from qiskit import Aer
from qiskit.providers.aer import AerSimulator
#from qiskit.algorithms import QAOA
from qiskit.aqua.algorithms import QAOA
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes,TwoLocal
#from qiskit.utils import QuantumInstance
from qiskit.aqua import QuantumInstance

from docplex.mp.model import Model
from qiskit_optimization import QuadraticProgram
from qiskit import IBMQ
from qiskit.optimization.applications.ising import docplex
#

import logging
import numpy as np
import pandas as pd
import math


Important prelim functions

In [4]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [5]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length)).astype(int)
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Feasibility functions

In [6]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n)).astype(int)
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [7]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    
    feasible=False
    while feasible==False and counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        frequency = bestinarray[counter][1]
        counter += 1
        
    if feasible == False:
        return feasible
    else:
        return bestasint[1:], frequency/total

## IBM device loading

In [8]:
from qiskit import IBMQ

# Load account from disk
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='wits-eie')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='reservations', project='reservations')>]

In [9]:
provider = IBMQ.get_provider(hub='ibm-q-wits',project='physics')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_manhattan') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_casab

## The QAOA function

In [17]:
def testing_quantum(ins, backend, optimal_point, trials):
    """
    Input:
    ins: name of instance
    backend
    optimal_point
    trials
    
    Output:
    outmatrix : [soln,result time,eigenvector,number of spsa trials]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #QAOA
    machinesolutions = []
    rmachinesolutions = []
    seed = 10598
    mod = QuadraticProgram()
    mod.from_docplex(mdl)
    #print(mod.export_as_lp_string())

    #30 trials
    ans = np.zeros(5)
    for i in [1,10,100,1000]:
    #try:
        file = open("thirty_trials-" + str(backend) + str(i) + "-" + str(ins) ,"w")
        file.write("value,   feasible,   frequency, time, iteration" + "\n")
        spsa = SPSA(maxiter=i)
        Backend = AerSimulator(method='matrix_product_state')
        #Backend = provider.get_backend(backend)
        quantum_instance = QuantumInstance(backend=Backend,seed_simulator=seed,
                                           seed_transpiler=seed, skip_qobj_validation = False, shots = 8192)

        qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=1,include_custom=True)#,initial_point = optimal_point)
        result = qaoa.run(quantum_instance)
        print(result['optimal_point'])
        #print('QAOA vector:', result['eigenstate'])
        print('QAOA time:', result['optimizer_time'])
        print('QAOA QAP objective:', result['eigenvalue'] + offset_docplex)

        n = len(list(result['eigenstate'].values()))
        solution = np.hstack((np.array(list(result['eigenstate'].values())).reshape(n,1),np.array(list(result['eigenstate'].keys())).reshape(n,1)))
        print(solution)
        for r in solution:
            file.write(str(qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc)) + "," + str(qap_feasible(np.array(list(r[1])))) + "," + str(r[0]) + ","+ str(result['optimizer_time']) + "," + str(i) + "\n")
            ans = np.vstack((ans,np.array([qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc),qap_feasible(np.array(list(r[1]))),float(r[0]),result['optimizer_time'],i])))
        #except:
            #file.write("An error occurred on iteration " + str(i) + " of the 30 trials")
            #print("AN ERROR OCCURRED ON ITERATION" + str(i) + " of the 30 trials")
            #ans = np.vstack((ans,np.array([math.inf,False,0,np.nan,i])))
        print("Iteration "+ str(i) + " is complete.")
        file.close()
    return ans[1:] 
    

In [18]:
def read_optimal(ins):
    data = pd.read_csv("initial_point-" + str(ins),header = None)
    n = len(data[0]) - 1
    ans = []
    data[0][0] = data[0][0][1:]
    data[0][n] = data[0][n][:-1]
    for i in data[0]:
        r = i.split(" ")
        for t in r:
            if t!='':
                ans.append(float(t))
    return np.array(ans)

## The results cells

In [ ]:
ins = "made3.csv"
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
backend = 'ibmq_qasm_simulator'
optimal_point = read_optimal(ins)
ans = testing_quantum(ins, backend, optimal_point, 30)

INFO:qiskit.aqua.quantum_instance:
Qiskit Terra version: 0.17.4
Backend: 'aer_simulator_matrix_product_state (None)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cswap', 'csx', 'cx', 'cy', 'cz', 'delay', 'diagonal', 'h', 'id', 'initialize', 'p', 'r', 'roerror', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z', 'kraus', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_density_matrix', 'save_expval', 'save_expval_var', 'save_matrix_product_state', 'save_probabilities', 'save_probabilities_dict', 'save_state', 'save_statevector', 'set_matrix_product_state', 'snapshot'], 'coupling_map': None}
{'initial_layout': None, 'seed_transpiler': 10598, 'optimization_level': None}
RunConfig(max_credits=10, seed_simulator=10598, shots=8192)
{'timeout': None}
{}
{}
Measurement mitigation: None
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:
==================== Setting of QAOA =================

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 0.20623 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('rx', 1), ('rz', 1), ('cx', 2), ('h', 1), ('measure', 1)} to target basis {'save_statevector', 'ry', 'u1', 'cswap', 'rz', 'y', 's', 'delay', 'h', 'snapshot', 'p', 'save_expval', 'rzz', 'x', 'save_state', 'save_amplitudes', 'u2', 't', 'kraus', 'cz', 'reset', 'roerror', 'unitary', 'save_amplitudes_sq', 'csx', 'tdg', 'cx', 'set_matrix_product_state', 'rxx', 'z', 'ccx', 'rx', 'sx', 'save_density_matrix', 'sdg', 'initialize', 'cy', 'ryy', 'cp', 'diagonal', 'barrier', 'measure', 'save_probabilities_dict', 'rzx', 'swap', 'save_probabilities', 'save_matrix_product_state', 'u', 'u3', 'save_expval_var', 'r', 'id'}.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Begining basis search from frozenset({('h', 1), ('rx', 1), ('rz', 1), ('measure', 1), ('cx', 2)}) to frozenset({'save_statevector', 'ry', 'u1', 'cswap'

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.07296 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [11541.16577148] - 1287.74643 (ms), eval count: 3
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.87590 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.09274 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-4343.79833984] - 1123.01230 (ms), eval count: 4
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.11439 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06652 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [30507.85400391] - 1079.44775 (ms), eval count: 5
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.07577 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.10777 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-13

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.12660 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-4343.79833984] - 1094.04778 (ms), eval count: 32
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.27604 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04625 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-13311.89086914] - 1079.52785 (ms), eval count: 33
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.61411 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.14997 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [30507.85400391] - 1084.59210 (ms), eval count: 34
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.57120 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.15759 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned 

...

INFO:qiskit.aqua.algorithms.vq_algorithm:Starting optimizer.
bounds=[(None, None), (0, 6.283185307179586)]
initial point=[3.7081342  3.23097276]


[-1.31816058  2.70554755]
QAOA time: 59.78340220451355
QAOA QAP objective: (524278.4328613281+0j)
[['2' '000110011']
 ['1' '001111100']
 ['1' '111010010']
 ['1' '000000111']
 ['1' '100100110']
 ['3' '100110001']
 ['1' '110001111']
 ['2' '110010001']
 ['3' '111010111']
 ['1' '010101001']
 ['2' '111100100']
 ['2' '011100100']
 ['2' '110101101']
 ['4' '001010000']
 ['6' '110000010']
 ['4' '011010000']
 ['4' '111001000']
 ['1' '001110101']
 ['3' '011100110']
 ['3' '001110000']
 ['2' '000101000']
 ['2' '010011011']
 ['4' '001011000']
 ['3' '010110000']
 ['8' '111011001']
 ['1' '100101110']
 ['1' '011001011']
 ['2' '101010110']
 ['1' '100011010']
 ['3' '101000010']
 ['17' '101101110']
 ['3' '110111000']
 ['6' '001101011']
 ['5' '110010100']
 ['10' '111110010']
 ['10' '101010101']
 ['4' '111110111']
 ['9' '010111011']
 ['5' '101110000']
 ['5' '110111111']
 ['3' '101000001']
 ['3' '010011110']
 ['9' '001110011']
 ['5' '000110010']
 ['9' '010101110']
 ['8' '011100010']
 ['19' '001000010']
 ['6'

INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 0.20099 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('rx', 1), ('rz', 1), ('cx', 2), ('h', 1), ('measure', 1)} to target basis {'save_statevector', 'ry', 'u1', 'cswap', 'rz', 'y', 's', 'delay', 'h', 'snapshot', 'p', 'save_expval', 'rzz', 'x', 'save_state', 'save_amplitudes', 'u2', 't', 'kraus', 'cz', 'reset', 'roerror', 'unitary', 'save_amplitudes_sq', 'csx', 'tdg', 'cx', 'set_matrix_product_state', 'rxx', 'z', 'ccx', 'rx', 'sx', 'save_density_matrix', 'sdg', 'initialize', 'cy', 'ryy', 'cp', 'diagonal', 'barrier', 'measure', 'save_probabilities_dict', 'rzx', 'swap', 'save_probabilities', 'save_matrix_product_state', 'u', 'u3', 'save_expval_var', 'r', 'id'}.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Begining basis search from frozenset({('h', 1), ('rx', 1), ('rz', 1), ('measure', 1), ('cx', 2)}) to frozenset({'save_statevector', 'ry', 'u1', 'cswap'

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04935 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [4767.4074707] - 1229.97069 (ms), eval count: 3
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 7.63035 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.12851 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [6606.66479492] - 1329.27704 (ms), eval count: 4
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 8.39853 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.09131 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [6606.66479492] - 1684.67450 (ms), eval count: 5
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 6.37817 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06843 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [4767.40

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04959 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [3101.24145508] - 1102.70953 (ms), eval count: 32
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.13013 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04745 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [6606.66479492] - 1128.00622 (ms), eval count: 33
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.15492 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05054 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [4767.4074707] - 1090.67535 (ms), eval count: 34
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.65584 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06890 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [4767

In [ ]:
"""runtime"""

In [14]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.17.4', 'qiskit-aer': '0.8.2', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': '0.13.1', 'qiskit-aqua': '0.9.1', 'qiskit': '0.26.2', 'qiskit-nature': '0.1.3', 'qiskit-finance': None, 'qiskit-optimization': '0.1.0', 'qiskit-machine-learning': None}